In [44]:
import torch
from PIL import Image
import json
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
#https://github.com/salesforce/LAVIS
from lavis.models import load_model_and_preprocess
from utils.coco_utils.annotate_images import annotate_img
from time import time
import os 
import torch.nn.functional as F

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('loading model')
model, vis_processors, txt_processors  = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device)


loading model


c:\Users\neals\anaconda3\envs\practicum\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [46]:
sample = {"image": None, "text_input": ['A sattellite image of a plane']}
sample2 = {"image": None, "text_input": ['A sattellite image of a ship']}
features_text1 = model.extract_features(sample, mode="text")
features_text2 = model.extract_features(sample2, mode="text")

In [47]:
features_text1

BlipOutputFeatures(image_embeds=None, image_embeds_proj=None, text_embeds=tensor([[[-0.0419,  0.0328,  0.1032,  ...,  0.0038,  0.3726,  0.1698],
         [-0.2967,  0.0822,  0.4625,  ..., -0.0868,  0.8165,  0.4379],
         [ 0.3968,  0.0216,  0.9961,  ..., -0.0717,  0.6900,  0.5643],
         ...,
         [-0.4789, -0.3678,  0.7024,  ..., -0.2755,  0.8651,  0.5216],
         [-0.1453, -0.0287,  0.4917,  ..., -0.4297,  1.7845,  0.1681],
         [ 0.0681,  0.3537, -0.0088,  ...,  0.5928,  0.1952, -0.1014]]],
       device='cuda:0'), text_embeds_proj=tensor([[[-0.0088, -0.0744, -0.0756,  ..., -0.0123, -0.0072,  0.0774],
         [ 0.0232, -0.1133, -0.0526,  ...,  0.0543, -0.0306,  0.0679],
         [ 0.0421, -0.0692, -0.1240,  ...,  0.0159, -0.0146, -0.0422],
         ...,
         [ 0.0231, -0.0722, -0.0547,  ...,  0.0809, -0.0653,  0.0302],
         [ 0.1010, -0.0583, -0.0315,  ..., -0.0014, -0.0018,  0.0293],
         [-0.1161,  0.0036, -0.0813,  ..., -0.1057,  0.0155,  0.1073]]],


In [48]:
features_text1.text_embeds_proj[:,0,:].t().shape

torch.Size([256, 1])

In [49]:
#similarity = (features_image.image_embeds_proj @ features_text.text_embeds_proj[:,0,:].t()).max()
#similarity = F.cosine_similarity(features_text1.text_embeds_proj[:,0,:].t(), features_text2.text_embeds_proj[:,0,:].t())
similarity = F.cosine_similarity(features_text1.text_embeds_proj[:,0,:], features_text2.text_embeds_proj[:,0,:])
print(similarity)  # This should give a scalar value
euclidean_dist = torch.norm(features_text1.text_embeds_proj[:,0,:].t() - features_text2.text_embeds_proj[:,0,:].t())

print(euclidean_dist)  

tensor([0.7799], device='cuda:0')
tensor(0.6635, device='cuda:0')


In [50]:
features_text1.text_embeds_proj[:,0,:].shape

torch.Size([1, 256])

In [52]:
sample = {"image": None, "text_input": ['''The image appears to be an aerial or satellite view of a landscape that includes various elements. Here's a detailed description of each visible item:
1. **Buildings**: There are several structures visible in the image. On the top left, there is a large building with a gray roof. Below this, there are two smaller structures, one with a gray roof and another with a darker, possibly brown roof.
2. **Vehicles**: There are a few vehicles visible. Near the bottom of the image, there is a white vehicle and a darker colored vehicle parked next to a building.
3. **Roads and Pathways**: A road or pathway runs vertically down the center of the image, leading to the buildings and parking areas.
4. **Vegetation**: The area is heavily vegetated with various shades of green, indicating different types of trees and possibly some grassy areas. There is a dense cluster of trees in the center of the image.
5. **Bare Land**: There is a large patch of bare land or possibly a field with very short vegetation in the center, surrounded by the trees.
6. **Miscellaneous Debris or Materials**: Scattered around the buildings and open land are various patches that could be debris, materials, or disturbed soil.
This image provides a comprehensive view of a rural or semi-rural property with residential and possibly agricultural or storage uses.''']}
sample2 = {"image": None, "text_input": ['''Describe every item in the image,"The image appears to be an aerial view of a residential area with various items visible. Here's a description of each item seen in the image:
1. **Houses**: There are two houses visible, one with a gray roof and another with a brown roof.
2. **Boat**: A small boat is docked on the water near a pier.
3. **Pier**: A small pier extends into the water, with a blue and white structure at the end.
4. **Garden or Flower Bed**: There is a colorful area that looks like a garden or flower bed near the house with the gray roof.
5. **Trees and Shrubs**: Various trees and shrubs are scattered throughout the yard areas, providing greenery and shade.
6. **Lawn Areas**: There are open grassy areas around the houses.
7. **Driveway**: A driveway is visible leading up to the house with the brown roof.
8. **Outdoor Structures**: There is a small, round, white structure that could be a storage shed or a playhouse in the yard of the house with the gray roof.
9. **Fence or Boundary**: A line of bushes or a low fence appears to demarcate the boundary between the two properties.
These elements together create a residential scene with typical backyard features.''']}
sample3 = {"image": None, "text_input": ['''The image appears to be a satellite view of a ship in a harbor''']}
features_text1 = model.extract_features(sample, mode="text")
features_text2 = model.extract_features(sample2, mode="text")
features_text3 = model.extract_features(sample3, mode="text")
similarity = F.cosine_similarity(features_text1.text_embeds_proj[:,0,:], features_text2.text_embeds_proj[:,0,:])
print(similarity)  # This should give a scalar value
euclidean_dist = torch.norm(features_text1.text_embeds_proj[:,0,:].t() - features_text2.text_embeds_proj[:,0,:].t())
print(euclidean_dist) 

similarity = F.cosine_similarity(features_text1.text_embeds_proj[:,0,:], features_text3.text_embeds_proj[:,0,:])
print(similarity)  # This should give a scalar value
euclidean_dist = torch.norm(features_text1.text_embeds_proj[:,0,:].t() - features_text3.text_embeds_proj[:,0,:].t())
print(euclidean_dist)  
 
similarity = F.cosine_similarity(features_text2.text_embeds_proj[:,0,:], features_text3.text_embeds_proj[:,0,:])
print(similarity)  # This should give a scalar value
euclidean_dist = torch.norm(features_text2.text_embeds_proj[:,0,:].t() - features_text3.text_embeds_proj[:,0,:].t())
print(euclidean_dist)  

tensor([0.8689], device='cuda:0')
tensor(0.5120, device='cuda:0')
tensor([0.5677], device='cuda:0')
tensor(0.9298, device='cuda:0')
tensor([0.5126], device='cuda:0')
tensor(0.9873, device='cuda:0')
